In [2]:
!pip install --upgrade pip
!pip install gcsfs
!pip install spacy

Requirement already up-to-date: pip in /opt/conda/lib/python3.7/site-packages (20.0.2)
     |████████████████████████████████| 10.6 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 185 kB 65.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 53.3 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 70.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.7 MB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 1.4 MB/s eta 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.27.2
    Uninstalling grpcio-1.27.2:
      Successfully uninstalled grpcio-1.27.2
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '_base_call.py'
Consider using the `--user` option or check the permissions.

     |████████████████████████████████| 421.8 MB 15 kB/s s eta 0:00:01
     |████████████████████████████████| 3.8 MB 51.5 MB/s eta 0:00:01
     |█████████████████████████████

In [3]:
%%time
import os, sys, time, json, re, string, datetime 
import multiprocessing as mp
import pandas as pd
import numpy as np
import spacy
from google.cloud import storage

import tensorflow as tf
import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, MaxPooling1D, CuDNNLSTM, Conv1D, CuDNNGRU

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 16} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

Using TensorFlow backend.


CPU times: user 2.62 s, sys: 1.05 s, total: 3.67 s
Wall time: 11.6 s


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
%%time
%%time
df_fake_train = pd.read_csv('gs://weicheng30417/data/fake-news/100k_fake_news_cleaned_dataset.csv')
df_fake_train[['authors_missing']] = df_fake_train[['authors_missing']].astype(int)
df_fake_train['label'] = 1
df_reliable_train = pd.read_csv('gs://weicheng30417/data/fake-news/100k_reliable_news_cleaned_dataset.csv')
df_reliable_train[['authors_missing']] = df_reliable_train[['authors_missing']].astype(int)
df_reliable_train['label'] = 0
df_train = pd.concat([df_fake_train, df_reliable_train])
df_train[df_train.columns] = df_train[df_train.columns].astype(str)
print(df_train.info())
df_train

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217806 entries, 0 to 108010
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   domain           217806 non-null  object
 1   type             217806 non-null  object
 2   content          217806 non-null  object
 3   title            217806 non-null  object
 4   authors          217806 non-null  object
 5   authors_missing  217806 non-null  object
 6   label            217806 non-null  object
dtypes: object(7)
memory usage: 13.3+ MB
None
CPU times: user 8.23 s, sys: 1.86 s, total: 10.1 s
Wall time: 30.4 s
CPU times: user 8.24 s, sys: 1.86 s, total: 10.1 s
Wall time: 30.4 s


,domain,type,content,title,authors,authors_missing,label
0,beforeitsnews.com,fake,Boehner presses to make tax cuts permanent\n\n...,Boehner presses to make tax cuts permanent,United Liberty,0,1
1,beforeitsnews.com,fake,An Armed Good Samaritan Who Doesn’t Want The S...,An Armed Good Samaritan Who Doesn’t Want The S...,The Real Revo,0,1
2,beforeitsnews.com,fake,(Before It's News)\n\nby Rob Morphy\n\nLegends...,Village Of The Dead: The Anjikuni Mystery,"Rob Morphy, Mort Amsel",0,1
3,teaparty.org,fake,(Breitbart) – With his hysterical gotcha attac...,Trump Calls Out Race-Baiting ABC News Reporter,nan,1,1
4,beforeitsnews.com,fake,Researchers develop new depression diagnosis a...,Researchers develop new depression diagnosis a...,Bel Marra Health,0,1
...,...,...,...,...,...,...,...
108006,sports.yahoo.com,reliable,"View photos\nMichigan guard Zak Irvin, right, ...",No. 25 Michigan beats Mount St. Mary's 64-47,The Associated Press,0,0
108007,uk.finance.yahoo.com,reliable,President-elect Donald Trump continued his scr...,"Trump quotes Hillary Clinton, rages against Wi...",Maxwell Tani,0,0
108008,www.yahoo.com,reliable,Holiday shoppers eager to snag big discounts t...,Dialing up deals: Black Friday online sales hi...,ALEX VEIGA,0,0
108009,www.reuters.com,reliable,Kabul police raid shisha cafes in crackdown on...,Kabul police raid shisha cafes in crackdown on...,nan,1,0


In [5]:
%%time
max_words = 3000
tk = Tokenizer(lower = True)
tk.fit_on_texts(df_train.content)
df_train_seq = tk.texts_to_sequences(df_train.content)
df_train_pad = pad_sequences(df_train_seq, maxlen=max_words, padding='post')

vocabulary_size = len(tk.word_counts.keys())+1
X_train, X_test, y_train, y_test = train_test_split(df_train_pad, df_train.label, test_size = 0.2, random_state = 1)
X_train_data, X_validate_data, y_train_data, y_validate_data = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1)

CPU times: user 3min 27s, sys: 2.25 s, total: 3min 30s
Wall time: 3min 30s


In [6]:
model = Sequential()
model.add(Embedding(vocabulary_size, 150, input_length=max_words))
model.add(Bidirectional(CuDNNLSTM(256)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_data, y_train_data, validation_data=(X_validate_data, y_validate_data), batch_size=64, epochs=5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:421: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 102263250 elements. This may consume a large amount of memory.
  num_elements)



Train on 139395 samples, validate on 34849 samples
Epoch 1/5
139395/139395 [==============================] - 1116s 8ms/step - loss: 0.0827 - accuracy: 0.9672 - val_loss: 0.0686 - val_accuracy: 0.9731
Epoch 2/5
139395/139395 [==============================] - 1106s 8ms/step - loss: 0.0259 - accuracy: 0.9908 - val_loss: 0.0559 - val_accuracy: 0.9811
Epoch 3/5
139395/139395 [==============================] - 1115s 8ms/step - loss: 0.0114 - accuracy: 0.9962 - val_loss: 0.0758 - val_accuracy: 0.9782
Epoch 4/5
139395/139395 [==============================] - 1112s 8ms/step - loss: 0.0079 - accuracy: 0.9976 - val_loss: 0.0718 - val_accuracy: 0.9779
Epoch 5/5
139395/139395 [==============================] - 1113s 8ms/step - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.0914 - val_accuracy: 0.9800


In [7]:
score = model.evaluate(X_test, y_test, verbose = 0)
print('Test accuracy:', score[1])

Test accuracy: 0.9804187417030334


In [8]:
import joblib
joblib.dump(model,  'fakenews_rnn_model.joblib')

['fakenews_rnn_model.joblib']